<a href="https://colab.research.google.com/github/Rauan2308/Empirically-assessing-the-predicitivty-of-tourism-demand-data/blob/main/Predictivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"Preparing data for the experimental part"
# Download the excel file from ABS: Table 11: Short-term movement, visitors arriving - state of intended stay: original
# In the first row keep only the destination names: Total, NSW, Vic, Qld, SA, WA, Tas, NT, ACT
# Delete the rows: Unit, Series Type, .. up to Series ID
# Add COVID column with dummy variables: 0 - from Jan 1991 to Jan 2020, 1 - from Feb 2020 to Oct 2024
# Upload the file using the code below

from google.colab import files
uploaded = files.upload()

!pip install openpyxl  # Install openpyxl if needed
import pandas as pd
import matplotlib.pyplot as plt

import io
file_name = list(uploaded.keys())[0]  # Get the uploaded file name

# Read Excel file
df = pd.read_excel(io.BytesIO(uploaded[file_name]), engine='openpyxl')

# Ensure the first column is treated as dates
df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], errors='coerce')

# Set the first column as the index
df.set_index(df.columns[0], inplace=True)

# Display the first few rows
print(df.head())

In [ ]:
"Calculation of Sample Entropy and MultiSampEn"

!pip install pyentrp
from scipy.optimize import fsolve
from cmath import log
from pyentrp.entropy import multiscale_entropy

# Function to calculate phi_max for each entropy value
def phi_max(s_values, length): #
    phi_values = []
    for s in s_values:
        func = lambda x: (-(x * log(x, 2).real + (1 - x) * log(1 - x, 2).real) + (1 - x) * log(length - 1, 2).real) - s
        ub = fsolve(func, 0.1)[0]  # Solve for upper bound
        phi_values.append(ub)
    return phi_values  # Return phi_max for each sub-series

# Sample Entropy Calculation
def entropy_cal(input_data, length, order):
    X_10 = []
    S10_s1 = []
    n = length

    # Create sub-series
    for index in range(len(input_data) - n + 1):
        X_10.append(input_data[index: index + n])

    # Calculate Sample Entropy (SampEn) for each sub-series
    for sub_series in X_10:
        sampen = ent.multiscale_entropy(sub_series, order, 0.65 * np.std(sub_series), 6)
        sampen = [val / np.log(2) for val in sampen]  # Convert to log base 2
        S10_s1.append(sampen[0])  # Store only the first scale SampEn value, change to store a required scale

    return S10_s1  # Return all entropy values for sub-series

In [ ]:
"Calculation of WPE and Predictivity(WPE)"

from pyentrp.entropy import weighted_permutation_entropy

# Function to extract subseries
def subseries_cal(input_series, length):
    sub_series_list = []
    for index in range(len(input_series) - length + 1):
        sub_series = input_series[index: index + length]
        sub_series_list.append(sub_series)
    return sub_series_list

# WPE Calculation
def pred_wpe(input_series, order, delay):
    return weighted_permutation_entropy(input_series, order, delay, normalize=False)

# Predictivity(WPE) Calculation
def pred_wpe(input_series, order, delay):
    return 1 - weighted_permutation_entropy(input_series, order, delay, normalize=True)

In [ ]:
"Calculation of RMSPE"

def rmspe(y_true, y_pred):
    """Calculate the root mean squared percentage error (RMSPE)."""
    if np.any(y_true == 0):
        return np.nan  # Or a large value, or handle this appropriately
    if len(y_true) == 0 or len(y_pred) == 0:  # Handle empty input arrays
        return np.nan
    if len(y_true) != len(y_pred):
        print("Error: y_true and y_pred have different lengths in RMSPE calculation.")
        return np.nan
    percentage_errors = np.abs((y_true - y_pred) / y_true)
    mse = np.mean(percentage_errors**2)
    return np.sqrt(mse)

In [ ]:
"Seasonal-Trend decomposition using STL"

!pip install pyentrp pyts
from statsmodels.tsa.seasonal import seasonal_decompose

def sstl_decomposition(series):
    series = pd.Series(series)
    stl = seasonal_decompose(series, model='additive', period=24, extrapolate_trend='freq')
    stl_trend_seasonal = stl.trend.fillna(method="bfill").fillna(method="ffill")+stl.seasonal
    return stl_trend_seasonal

In [ ]:
"Singular Spectrum Analysis (SSA) decomposition and sum the first `num_components` components."

from pyts.decomposition import SingularSpectrumAnalysis as SSA

def ssa_decomposition(series, num_components=2):
    series = np.array(series).reshape(1, -1)
    ssa = SSA(window_size=12)
    X_s = ssa.fit_transform(series)
    num_components = min(num_components, X_s.shape[1])
    reconstructed = np.sum(X_s[:, :num_components], axis=1)
    return reconstructed.flatten()

In [ ]:
"ARIMA model"

import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Function to find the best ARIMA order
def find_best_arima(series, p_range=(0, 3), d_range=(0, 3), q_range=(0, 3)):
    best_aic = float("inf")
    best_order = None
    for p, d, q in product(range(*p_range), range(*d_range), range(*q_range)):
        try:
            model = ARIMA(series, order=(p, d, q))
            model_fit = model.fit()
            if model_fit.aic < best_aic:
                best_aic = model_fit.aic
                best_order = (p, d, q)
        except:
            continue
    return best_order

# Function to forecast using ARIMA
def arima_forecast(series, forecast_horizon=12):
    series = np.array(series)
    train, test = series[:-forecast_horizon], series[-forecast_horizon:]
    best_order = find_best_arima(train)
    if best_order is None:
        print("Warning: No suitable ARIMA model found.")
        return np.full(forecast_horizon, np.nan), test
    model = ARIMA(train, order=best_order)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=forecast_horizon)
    return forecast, test

In [ ]:
"Kendall's and Spearman's correlations"

from scipy.stats import kendalltau, spearmanr

# Function to compute correlation scores
def compute_correlations(data):
    kendall_rmspe_predictivity, kendall_p_predictivity = stats.kendalltau(data["RMSPE"], data["Predictivity"])
    spearman_rmspe_predictivity, spearman_p_predictivity = stats.spearmanr(data["RMSPE"], data["Predictivity"])

    kendall_rmspe_phi, kendall_p_phi = stats.kendalltau(data["RMSPE"], data["Phi_max"])
    spearman_rmspe_phi, spearman_p_phi = stats.spearmanr(data["RMSPE"], data["Phi_max"])

    results = {
        "Kendall's Tau (RMSPE vs Predictivity)": kendall_rmspe_predictivity,
        "Kendall's p-value (RMSPE vs Predictivity)": kendall_p_predictivity,
        "Spearman's Rho (RMSPE vs Predictivity)": spearman_rmspe_predictivity,
        "Spearman's p-value (RMSPE vs Predictivity)": spearman_p_predictivity,
        "Kendall's Tau (RMSPE vs Phi_max)": kendall_rmspe_phi,
        "Kendall's p-value (RMSPE vs Phi_max)": kendall_p_phi,
        "Spearman's Rho (RMSPE vs Phi_max)": spearman_rmspe_phi,
        "Spearman's p-value (RMSPE vs Phi_max)": spearman_p_phi
    }

    return pd.DataFrame([results])

In [ ]:
"WPE calculation for tuning"

!pip install pyentrp
# prepare the library on entropy calculation

import numpy as np
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.dates import YearLocator, DateFormatter
from pyentrp import entropy as ent
import seaborn as sns
from scipy.optimize import fsolve
from cmath import log
from pylab import rcParams
from statsmodels.tsa.seasonal import seasonal_decompose
from pyentrp.entropy import weighted_permutation_entropy
import numpy as np
import seaborn as sns
from scipy.optimize import fsolve
from math import log

X=df.Total.values.reshape(406,)

def calculate_wpe_for_combinations(X, orders, delays):
    results = []

    # Iterate over all combinations of order and delay
    for order in orders:
        for delay in delays:
            # Calculate WPE for the current order and delay
            wpe = weighted_permutation_entropy(X, order, delay, normalize=False)
            results.append((order, delay, wpe))

    return results

orders = range(3,5)
delays = range(1,21)

# Calculate WPE
phi_results = calculate_wpe_for_combinations(X, orders, delays)

# Convert results to a DataFrame for easy plotting
phi_results = pd.DataFrame(phi_results, columns=['Order', 'Delay', 'WPE'])

# Create the line plot
plt.figure(figsize=(10, 6))
sns.set(style="whitegrid")

for order in orders:
    subset = phi_results[phi_results['Order'] == order]
    plt.plot(subset['Delay'], subset['WPE'], label=f'm = {order}')

plt.xticks(ticks=delays)
plt.grid(False)

plt.xlabel(f'Time delay, $\\tau$')
plt.ylabel('WPE')
plt.title('WPE for embedding dimension $m=3,4$ and time delay $\\tau = 1-20$')
plt.legend(title='Embedding dimension', loc='upper center', bbox_to_anchor=(0.5, -0.13), ncol=3)
plt.tight_layout()
plt.show()

In [ ]:
"Predictivity(WPE) calculation for tuning"

def calculate_wpe_for_combinations(X, orders, delays):
    results = []

    # Iterate over all combinations of order and delay
    for order in orders:
        for delay in delays:
            # Calculate WPE for the current order and delay
            phi = 1 - weighted_permutation_entropy(X, order, delay, normalize=True)
            results.append((order, delay, phi))

    return results

orders = range(3,5)
delays = range(1,21)

# Calculate WPE
wpe_results = calculate_wpe_for_combinations(X, orders, delays)

# Convert results to a DataFrame for easy plotting
df_ = pd.DataFrame(wpe_results, columns=['Order', 'Delay', 'WPE'])

# Create the line plot
plt.figure(figsize=(10, 6))
sns.set(style="whitegrid")

for order in orders:
    subset = df_[df_['Order'] == order]
    plt.plot(subset['Delay'], subset['WPE'], label=f'm = {order}')

plt.xticks(ticks=delays)
plt.grid(False)

plt.xlabel(f'Time delay, $\\tau$')
plt.ylabel('Predictivity')
plt.title('Predictivity for embedding dimension $m=3,4$ and time delay $\\tau = 1-20$')
plt.legend(title='Embedding dimension', loc='upper center', bbox_to_anchor=(0.5, -0.13), ncol=3)
plt.tight_layout()
plt.show()

In [ ]:
"WPE calculation by varying data length and scale"

#!pip install pyentrp

import matplotlib.pyplot as plt
import numpy as np
from pyentrp.entropy import weighted_permutation_entropy

#!pip install pyentrp

X = df.Total.values.reshape(406,)

# Define different resolutions
time_scales = {
    "monthly": (X, 12, 349),  # Original monthly data
    "quarterly": (X[:(406 // 3) * 3].reshape(-1, 3).mean(axis=1), 4, 116),  # Averaged quarterly data
    "half-yearly": (X[:(406 // 6) * 6].reshape(-1, 6).mean(axis=1), 2, 58)  # Averaged half-yearly data
    #"yearly": (X[:(406 // 12) * 12].reshape(-1, 12).mean(axis=1), 1, 29)  # Averaged yearly data
}

# Storage for results
results_f = {}
results_pre = {}
results_post = {}

# Function to extract subseries and COVID flags
def subseries_cal(input_series, length, post_covid_index):
    sub_series_list = []
    post_covid_flags = []

    for index in range(len(input_series) - length + 1):
        sub_series = input_series[index: index + length]
        sub_series_list.append(sub_series)

        # Determine pre- or post-COVID based on index
        post_covid_flags.append(1 if index + length - 1 >= post_covid_index else 0)

    return sub_series_list, post_covid_flags

# Function to calculate WPE
def entropy_cal(input_series, order, delay):
    wpe_values = [
        weighted_permutation_entropy(input_series[i: i + len(input_series)], order, delay, normalize=False)
        for i in range(len(input_series) - len(input_series) + 1)
    ]
    return np.mean(wpe_values) if wpe_values else 0

# Loop over time scales
for scale, (X_scaled, conversion_factor, post_covid_index) in time_scales.items():
    m_f_dict = {}
    m_pre_dict = {}
    m_post_dict = {}

    # Define appropriate length ranges
    min_length, max_length, step = {
        "monthly": (60, 406, 12),
        "quarterly": (32, 136, 4),
        "half-yearly": (32, 68, 2)
        #"yearly": (20, 34, 1)
    }[scale]

    lengths = range(min_length, max_length, step)
    orders = range(3, 4)
    delays = range(step, step + 1)

    # Calculate WPE for each length
    for length in lengths:
        sub_series_list, post_covid_flags = subseries_cal(X_scaled, length, post_covid_index)

        wpe_values = []
        wpe_pre = []
        wpe_post = []

        for i, sub_series in enumerate(sub_series_list):
            for order in orders:
                for delay in delays:
                    avg_wpe = entropy_cal(sub_series, order, delay)
                    wpe_values.append(avg_wpe)

                    if post_covid_flags[i] == 1:
                        wpe_post.append(avg_wpe)
                    else:
                        wpe_pre.append(avg_wpe)

        # Convert length to years
        length_in_years = length / conversion_factor

        # Store averages per length
        if wpe_values:
            m_f_dict[length_in_years] = np.mean(wpe_values)
        if wpe_pre:
            m_pre_dict[length_in_years] = np.mean(wpe_pre)
        if wpe_post:
            m_post_dict[length_in_years] = np.mean(wpe_post)

    # Store results for each time scale
    results_f[scale] = list(m_f_dict.items())
    results_pre[scale] = list(m_pre_dict.items())
    results_post[scale] = list(m_post_dict.items())

# Prepare the plot for full, pre-COVID, and post-COVID data
fig, axs = plt.subplots(2, 1, figsize=(10, 12))

# Plot full data
for scale, data in results_f.items():
    x_vals, y_vals = zip(*data)
    axs[0].plot(x_vals, y_vals, label=f'{scale} scale')

axs[0].set_title('Full time series')
axs[0].set_xlabel('Data Length (Years)')
axs[0].set_ylabel('WPE')
axs[0].legend(loc='best')
axs[0].grid(False)

# Set x-ticks with step 1 for full data, using np.arange for float values
x_vals_full = [x for scale, data in results_f.items() for x, _ in data]
axs[0].set_xticks(np.arange(min(x_vals_full), max(x_vals_full) + 1, 1))
axs[0].set_ylim(0.4, 2.1)  # Adjust y-axis limits (adjust as needed)

# Plot pre-COVID data
for scale, data in results_pre.items():
    x_vals, y_vals = zip(*data)
    axs[1].plot(x_vals, y_vals, label=f'{scale} time scale')

axs[1].set_title('Pre-COVID time series')
axs[1].set_xlabel('Data Length (Years)')
axs[1].set_ylabel('WPE')
#axs[1].legend(loc='best')
axs[1].grid(False)

# Set x-ticks with step 1 for pre-COVID data, using np.arange for float values
x_vals_pre = [x for scale, data in results_pre.items() for x, _ in data]
axs[1].set_xticks(np.arange(min(x_vals_pre), max(x_vals_pre) + 1, 1))
axs[1].set_ylim(0.4, 2.1)  # Adjust y-axis limits (adjust as needed)

In [ ]:
"Predictivity(WPE) calculation by varying data length and scale"

import matplotlib.pyplot as plt
import numpy as np
from pyentrp.entropy import weighted_permutation_entropy

X = df.Total.values.reshape(406,)

# Define different resolutions
time_scales = {
    "monthly": (X, 12, 349),  # Original monthly data
    "quarterly": (X[:(406 // 3) * 3].reshape(-1, 3).mean(axis=1), 4, 116),  # Averaged quarterly data
    "half-yearly": (X[:(406 // 6) * 6].reshape(-1, 6).mean(axis=1), 2, 58)  # Averaged half-yearly data
    #"yearly": (X[:(406 // 12) * 12].reshape(-1, 12).mean(axis=1), 1, 29)  # Averaged yearly data
}

# Storage for results
results_f = {}
results_pre = {}
results_post = {}

# Function to extract subseries and COVID flags
def subseries_cal(input_series, length, post_covid_index):
    sub_series_list = []
    post_covid_flags = []

    for index in range(len(input_series) - length + 1):
        sub_series = input_series[index: index + length]
        sub_series_list.append(sub_series)

        # Determine pre- or post-COVID based on index
        post_covid_flags.append(1 if index + length - 1 >= post_covid_index else 0)

    return sub_series_list, post_covid_flags

# Function to calculate WPE
def entropy_cal(input_series, order, delay):
    wpe_values = [
        1-weighted_permutation_entropy(input_series[i: i + len(input_series)], order, delay, normalize=True)
        for i in range(len(input_series) - len(input_series) + 1)
    ]
    return np.mean(wpe_values) if wpe_values else 0

# Loop over time scales
for scale, (X_scaled, conversion_factor, post_covid_index) in time_scales.items():
    m_f_dict = {}
    m_pre_dict = {}
    m_post_dict = {}

    # Define appropriate length ranges
    min_length, max_length, step = {
        "monthly": (60, 406, 12),
        "quarterly": (32, 136, 4),
        "half-yearly": (32, 68, 2)
        #"yearly": (20, 34, 1)
    }[scale]

    lengths = range(min_length, max_length, step)
    orders = range(3, 4)
    delays = range(step, step + 1)

    # Calculate WPE for each length
    for length in lengths:
        sub_series_list, post_covid_flags = subseries_cal(X_scaled, length, post_covid_index)

        wpe_values = []
        wpe_pre = []
        wpe_post = []

        for i, sub_series in enumerate(sub_series_list):
            for order in orders:
                for delay in delays:
                    avg_wpe = entropy_cal(sub_series, order, delay)
                    wpe_values.append(avg_wpe)

                    if post_covid_flags[i] == 1:
                        wpe_post.append(avg_wpe)
                    else:
                        wpe_pre.append(avg_wpe)

        # Convert length to years
        length_in_years = length / conversion_factor

        # Store averages per length
        if wpe_values:
            m_f_dict[length_in_years] = np.mean(wpe_values)
        if wpe_pre:
            m_pre_dict[length_in_years] = np.mean(wpe_pre)
        if wpe_post:
            m_post_dict[length_in_years] = np.mean(wpe_post)

    # Store results for each time scale
    results_f[scale] = list(m_f_dict.items())
    results_pre[scale] = list(m_pre_dict.items())
    results_post[scale] = list(m_post_dict.items())

# Prepare the plot for full, pre-COVID, and post-COVID data
fig, axs = plt.subplots(2, 1, figsize=(10, 12))

# Plot full data
for scale, data in results_f.items():
    x_vals, y_vals = zip(*data)
    axs[0].plot(x_vals, y_vals, label=f'{scale} time scale')

axs[0].set_title('Full time series')
axs[0].set_xlabel('Data Length (Years)')
axs[0].set_ylabel('Predictivity')
#axs[0].legend(loc='best')
axs[0].grid(False)

# Set x-ticks with step 1 for full data, using np.arange for float values
x_vals_full = [x for scale, data in results_f.items() for x, _ in data]
axs[0].set_xticks(np.arange(min(x_vals_full), max(x_vals_full) + 1, 1))
axs[0].set_ylim(0, 1)  # Adjust y-axis limits (adjust as needed)


# Plot pre-COVID data
for scale, data in results_pre.items():
    x_vals, y_vals = zip(*data)
    axs[1].plot(x_vals, y_vals, label=f'{scale} time series')

axs[1].set_title('Pre-COVID time series')
axs[1].set_xlabel('Data Length (Years)')
axs[1].set_ylabel('Predictivity')
#axs[1].legend(loc='best')
axs[1].grid(False)

# Set x-ticks with step 1 for pre-COVID data, using np.arange for float values
x_vals_pre = [x for scale, data in results_pre.items() for x, _ in data]
axs[1].set_xticks(np.arange(min(x_vals_pre), max(x_vals_pre) + 1, 1))
axs[1].set_ylim(0, 1)  # Adjust y-axis limits (adjust as needed)
